## Prepare environment

In [1]:
import pandas as pd
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn import metrics
import bokeh.plotting as bp
import itertools
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.tools.plotting import autocorrelation_plot
import datetime
from fbprophet import Prophet
from sklearn.preprocessing import MinMaxScaler

## Acquire

In [2]:
df = pd.read_csv('south_central_moisture_data_1988_2017.csv')
df.head()

,final_date,soil_moisture,pdsi
0,1988-03-01,4.79,0.0
1,1988-03-08,4.67,0.0
2,1988-03-15,4.24,0.0
3,1988-03-22,5.59,0.0
4,1988-03-29,5.00,0.0


## Scale data

In [3]:
df = df[['final_date', 'soil_moisture']]

mmscaler = MinMaxScaler(feature_range=(0, 1))
mmscaler.fit(df[['soil_moisture']])
df.soil_moisture = mmscaler.transform(df[['soil_moisture']])

df.head()

,final_date,soil_moisture
0,1988-03-01,0.598248
1,1988-03-08,0.583229
2,1988-03-15,0.529412
3,1988-03-22,0.698373
4,1988-03-29,0.624531


## Set index

In [4]:
df.final_date = pd.to_datetime(df.final_date)
df = df.set_index('final_date')

In [5]:
df_agg = df.soil_moisture.resample('W').agg('mean')

In [6]:
df_agg.head()

final_date
1988-03-06    0.598248
1988-03-13    0.583229
1988-03-20    0.529412
1988-03-27    0.698373
1988-04-03    0.624531
Freq: W-SUN, Name: soil_moisture, dtype: float64

## Only get data from 1997-present to match PDSI data

In [7]:
df_agg = df_agg['1997':]

In [8]:
df_agg = df_agg.to_frame(name=None)

In [9]:
df_agg.head()

,soil_moisture
final_date,
1997-01-05,0.446809
1997-01-12,0.515645
1997-01-19,0.528160
1997-01-26,0.658323
1997-02-02,0.639549


Compute soil moisture mean from weekly aggregate means.

In [10]:
agg_mean = df_agg.soil_moisture.mean()
agg_mean

0.4830032772573354

Computer soil moisture standard deviation from weekly aggregate means.

In [11]:
agg_stdev = df_agg.soil_moisture.std()
agg_stdev

0.35018915819138896

Compute mean-(4xstdev) to get lower bound

In [12]:
df_agg["lower_bound"] = (agg_mean - (4*agg_stdev))

In [13]:
df_agg.head()

,soil_moisture,lower_bound
final_date,,
1997-01-05,0.446809,-0.917753
1997-01-12,0.515645,-0.917753
1997-01-19,0.528160,-0.917753
1997-01-26,0.658323,-0.917753
1997-02-02,0.639549,-0.917753


In [14]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1105 entries, 1997-01-05 to 2018-03-04
Freq: W-SUN
Data columns (total 2 columns):
soil_moisture    1101 non-null float64
lower_bound      1105 non-null float64
dtypes: float64(2)
memory usage: 25.9 KB


There are four weeks missing data. These will be backfilled.

In [15]:
df_agg[df_agg.soil_moisture.isnull()]

,soil_moisture,lower_bound
final_date,,
1999-02-28,NaN,-0.917753
2004-02-29,NaN,-0.917753
2010-02-28,NaN,-0.917753
2016-02-28,NaN,-0.917753


In [16]:
df_agg.soil_moisture.fillna(method='bfill', inplace=True)

In [17]:
df_agg.soil_moisture.isnull().sum()

0

Created a column for "soil moisture - lower bound"

In [18]:
df_agg["sm_lb"] = (df_agg.soil_moisture - df_agg.lower_bound)

In [19]:
df_agg.head()

,soil_moisture,lower_bound,sm_lb
final_date,,,
1997-01-05,0.446809,-0.917753,1.364562
1997-01-12,0.515645,-0.917753,1.433398
1997-01-19,0.528160,-0.917753,1.445914
1997-01-26,0.658323,-0.917753,1.576076
1997-02-02,0.639549,-0.917753,1.557303


Created a boolean column for whether or not this value falls below normal range

In [20]:
df_agg['outside_normal'] = (df_agg.sm_lb < 1)

In [21]:
df_agg.tail()

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,
2018-02-04,0.824781,-0.917753,1.742534,False
2018-02-11,0.799750,-0.917753,1.717503,False
2018-02-18,0.813517,-0.917753,1.731270,False
2018-02-25,0.798498,-0.917753,1.716251,False
2018-03-04,0.874844,-0.917753,1.792597,False


In [22]:
df_agg.outside_normal.sum()

195

In [23]:
df_agg.soil_moisture.count()

1105

In [24]:
(195/1105)*100

17.647058823529413

In [25]:
# df_agg['2011':]

In [26]:
sm_non_normal = df_agg[df_agg.sm_lb < 1]

In [27]:
sm_non_normal['2017':]

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,


## PDSI df

In [28]:
pdsi = pd.read_csv('res_PDSI.csv')

In [29]:
pdsi.head()

,date,reservoir,PDSI
0,1997-09-30,285458.000000,2.86
1,1997-10-31,282507.709677,2.88
2,1997-11-30,277624.300000,2.71
3,1997-12-31,275259.387097,3.07
4,1998-01-31,272926.612903,3.02


In [30]:
pdsi = pdsi[['date', 'PDSI']]

In [31]:
pdsi.tail()

,date,PDSI
251,2018-08-31,-3.71
252,2018-09-30,1.50
253,2018-10-31,3.38
254,2018-11-30,3.58
255,2018-12-31,4.57


In [32]:
pdsi['drought'] = (pdsi.PDSI < (-2))

In [33]:
pdsi.sample(10)

,date,PDSI,drought
107,2006-08-31,-5.28,True
231,2016-12-31,-1.00,False
131,2008-08-31,-1.82,False
169,2011-10-31,-7.39,True
226,2016-07-31,4.70,False
118,2007-07-31,6.44,False
120,2007-09-30,-0.02,False
42,2001-03-31,4.14,False
151,2010-04-30,3.03,False
198,2014-03-31,-3.10,True


    Moderate Drought: PDSI < -2
    Severe Drought: PDSI < -3

In [34]:
drought = pdsi[pdsi.PDSI < (-2)]

In [35]:
drought.date = pd.to_datetime(drought.date)
drought = drought.set_index('date')

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [36]:
drought['2018':]

,PDSI,drought
date,,
2018-05-31,-2.94,True
2018-06-30,-3.50,True
2018-07-31,-3.77,True
2018-08-31,-3.71,True


In [37]:
pdsi.PDSI.count()

256

In [38]:
(85/256)*100

33.203125

## Soil moisture monthly aggregation

In [39]:
df_monthly = df.soil_moisture.resample('M').agg('mean')

In [40]:
df_monthly = df_monthly['1997':]

In [41]:
df_monthly = df_monthly.to_frame(name=None)

In [42]:
df_monthly.head()

,soil_moisture
final_date,
1997-01-31,0.557697
1997-02-28,0.766792
1997-03-31,0.948436
1997-04-30,0.989675
1997-05-31,0.977222


Compute soil moisture mean from monthly aggregate means.

In [43]:
monthly_mean = df_monthly.soil_moisture.mean()
monthly_mean

0.48451829550717923

Computer soil moisture standard deviation from monthly aggregate means.

In [44]:
monthly_stdev = df_monthly.soil_moisture.std()
monthly_stdev

0.33799572004510314

Compute mean-(4xstdev) to get lower bound

In [45]:
df_monthly["lower_bound"] = (monthly_mean - (4*monthly_stdev))

In [46]:
df_monthly.head()

,soil_moisture,lower_bound
final_date,,
1997-01-31,0.557697,-0.867465
1997-02-28,0.766792,-0.867465
1997-03-31,0.948436,-0.867465
1997-04-30,0.989675,-0.867465
1997-05-31,0.977222,-0.867465


In [47]:
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 254 entries, 1997-01-31 to 2018-02-28
Freq: M
Data columns (total 2 columns):
soil_moisture    254 non-null float64
lower_bound      254 non-null float64
dtypes: float64(2)
memory usage: 6.0 KB


Created a column for "soil moisture - lower bound"

In [48]:
df_monthly["sm_lb"] = (df_monthly.soil_moisture - df_monthly.lower_bound)

In [49]:
df_monthly.head()

,soil_moisture,lower_bound,sm_lb
final_date,,,
1997-01-31,0.557697,-0.867465,1.425162
1997-02-28,0.766792,-0.867465,1.634256
1997-03-31,0.948436,-0.867465,1.815900
1997-04-30,0.989675,-0.867465,1.857139
1997-05-31,0.977222,-0.867465,1.844686


Created a boolean column for whether or not this value falls below normal range

In [50]:
df_monthly['outside_normal'] = (df_monthly.sm_lb < 1)

In [51]:
df_monthly.head()

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,
1997-01-31,0.557697,-0.867465,1.425162,False
1997-02-28,0.766792,-0.867465,1.634256,False
1997-03-31,0.948436,-0.867465,1.815900,False
1997-04-30,0.989675,-0.867465,1.857139,False
1997-05-31,0.977222,-0.867465,1.844686,False


In [52]:
df_monthly.outside_normal.sum()

58

In [53]:
df_monthly.soil_moisture.count()

254

In [54]:
(58/254)*100

22.83464566929134

In [55]:
monthly_drought = df_monthly[df_monthly.sm_lb < 1]

In [62]:
df_monthly['2011':]

,soil_moisture,lower_bound,sm_lb,outside_normal,difference
final_date,,,,,
2011-01-31,0.576471,-0.867465,1.443935,False,0.227597
2011-02-28,0.770964,-0.867465,1.638428,False,0.194493
2011-03-31,0.479599,-0.867465,1.347064,False,-0.291364
2011-04-30,0.407071,-0.867465,1.274536,False,-0.072528
2011-05-31,0.237797,-0.867465,1.105262,False,-0.169274
2011-06-30,0.111389,-0.867465,0.978854,True,-0.126408
2011-07-31,0.051627,-0.867465,0.919092,True,-0.059762
2011-08-31,0.017522,-0.867465,0.884986,True,-0.034105
2011-09-30,0.005632,-0.867465,0.873097,True,-0.011890


In [56]:
monthly_drought['2010':]

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,
2011-06-30,0.111389,-0.867465,0.978854,True
2011-07-31,0.051627,-0.867465,0.919092,True
2011-08-31,0.017522,-0.867465,0.884986,True
2011-09-30,0.005632,-0.867465,0.873097,True
2011-10-31,0.013454,-0.867465,0.880919,True
2011-11-30,0.012516,-0.867465,0.879980,True
2012-08-31,0.125907,-0.867465,0.993372,True
2012-09-30,0.102315,-0.867465,0.969780,True
2012-10-31,0.072278,-0.867465,0.939742,True


In [57]:
df_monthly['difference'] = df_monthly.soil_moisture.diff()

In [58]:
df_monthly.sample(10)

,soil_moisture,lower_bound,sm_lb,outside_normal,difference
final_date,,,,,
2002-08-31,0.387985,-0.867465,1.255450,False,-0.349499
1997-06-30,0.994055,-0.867465,1.861520,False,0.016834
1997-09-30,0.154255,-0.867465,1.021720,False,-0.104318
1999-04-30,0.787234,-0.867465,1.654699,False,-0.008761
1998-05-31,0.468335,-0.867465,1.335800,False,-0.311076
2001-12-31,0.996871,-0.867465,1.864336,False,0.105257
2013-02-28,0.323321,-0.867465,1.190785,False,0.022695
2014-08-31,0.109637,-0.867465,0.977102,True,-0.145995
2000-06-30,0.357322,-0.867465,1.224786,False,0.012390


In [59]:
df_monthly.difference.max()

0.6725907384230287

In [60]:
df_monthly.difference.min()

-0.4339799749687109

In [61]:
df_monthly

,soil_moisture,lower_bound,sm_lb,outside_normal,difference
final_date,,,,,
1997-01-31,0.557697,-0.867465,1.425162,False,NaN
1997-02-28,0.766792,-0.867465,1.634256,False,0.209095
1997-03-31,0.948436,-0.867465,1.815900,False,0.181644
1997-04-30,0.989675,-0.867465,1.857139,False,0.041239
1997-05-31,0.977222,-0.867465,1.844686,False,-0.012453
1997-06-30,0.994055,-0.867465,1.861520,False,0.016834
1997-07-31,0.590738,-0.867465,1.458203,False,-0.403317
1997-08-31,0.258573,-0.867465,1.126038,False,-0.332165
1997-09-30,0.154255,-0.867465,1.021720,False,-0.104318
